In [ ]:
import pandas as pd
import os

def get_base_name(filename):
    """Extrai o nome base do arquivo, removendo sufixos _A, _B, _C e .xpt"""
    name = filename.replace('.xpt', '')
    if name.endswith('_A') or name.endswith('_B') or name.endswith('_C'):
        return name[:-2]
    return name

def find_common_datasets(base_paths):
    """Encontra todos os datasets que existem nos três ciclos"""
    datasets_by_cycle = {}
    
    # Coletar nomes base de cada ciclo
    for path in base_paths:
        cycle = os.path.basename(path)
        files = [f for f in os.listdir(path) if f.lower().endswith('.xpt')]
        datasets_by_cycle[cycle] = set(get_base_name(f) for f in files)
    
    # Encontrar interseção dos conjuntos
    common_datasets = set.intersection(*datasets_by_cycle.values())
    return sorted(list(common_datasets))

def merge_dataset(base_name, base_paths):
    """Combina um dataset específico dos três ciclos"""
    dfs = []
    
    for path in base_paths:
        cycle = os.path.basename(path)
        
        # Tentar encontrar o arquivo com o padrão correto
        if cycle == 'NHANES_1999_2000':
            possible_names = [f"{base_name}.xpt", f"{base_name}_A.xpt"]
        elif cycle == 'NHANES_2001_2002':
            possible_names = [f"{base_name}_B.xpt"]
        else:  # 2003-2004
            possible_names = [f"{base_name}_C.xpt"]
        
        file_found = False
        for file_name in possible_names:
            full_path = os.path.join(path, file_name)
            if os.path.exists(full_path):
                try:
                    print(f"Lendo {file_name}")
                    df = pd.read_sas(full_path)
                    
                    # Padronizar nomes das colunas
                    df.columns = [col[:-2] if col.endswith(('_A', '_B', '_C')) else col 
                                for col in df.columns]
                    
                    df['NHANES_CYCLE'] = cycle
                    dfs.append(df)
                    print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
                    file_found = True
                    break
                except Exception as e:
                    print(f"Erro ao ler {file_name}: {str(e)}")
        
        if not file_found:
            print(f"Nenhum arquivo encontrado para {base_name} em {cycle}")
            return None
    
    if len(dfs) == 3:
        # Encontrar colunas comuns
        common_cols = set.intersection(*[set(df.columns) for df in dfs]) - {'NHANES_CYCLE'}
        columns_to_keep = list(common_cols) + ['NHANES_CYCLE']
        
        # Combinar os datasets
        combined_df = pd.concat([df[columns_to_keep] for df in dfs], 
                              axis=0, ignore_index=True)
        print(f"Dataset combinado - Linhas: {combined_df.shape[0]}, Colunas: {combined_df.shape[1]}")
        return combined_df
    return None

# Caminhos para os diretórios
base_paths = [
    '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/NHANES_1999_2000',
    '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/NHANES_2001_2002',
    '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/NHANES_2003_2004'
]

# Criar pasta para outputs
output_dir = os.path.join(os.path.dirname(base_paths[0]), 'combined_datasets')
os.makedirs(output_dir, exist_ok=True)
print(f"Arquivos combinados serão salvos em: {output_dir}")

# Encontrar todos os datasets comuns
print("\nProcurando datasets comuns aos três ciclos...")
common_datasets = find_common_datasets(base_paths)
print(f"Encontrados {len(common_datasets)} datasets comuns:")
for dataset in common_datasets:
    print(f"- {dataset}")

# Processar cada dataset
print("\nIniciando processo de merge...")
successful_merges = []
failed_merges = []

for dataset in common_datasets:
    print(f"\nProcessando {dataset}...")
    combined_df = merge_dataset(dataset, base_paths)
    
    if combined_df is not None:
        output_path = os.path.join(output_dir, f'{dataset}_combined.csv')
        combined_df.to_csv(output_path, index=False)
        successful_merges.append(dataset)
        print(f"Dataset {dataset} salvo com sucesso")
    else:
        failed_merges.append(dataset)
        print(f"Falha ao combinar dataset {dataset}")

print("\nProcessamento concluído!")
print(f"\nDatasets combinados com sucesso ({len(successful_merges)}):")
for dataset in successful_merges:
    print(f"- {dataset}")

if failed_merges:
    print(f"\nDatasets que falharam ({len(failed_merges)}):")
    for dataset in failed_merges:
        print(f"- {dataset}")

In [ ]:
import pandas as pd
import os

def merge_panic_data(base_paths):
    """Combina os dados de pânico dos três ciclos"""
    dfs = []
    
    # Definir os nomes específicos para cada ciclo
    file_names = {
        'NHANES_1999_2000': 'CIQPANIC.xpt',
        'NHANES_2001_2002': 'CIQPAN_B.xpt',
        'NHANES_2003_2004': 'CIQPAN_C.xpt'
    }
    
    for path in base_paths:
        cycle = os.path.basename(path)
        file_name = file_names[cycle]
        full_path = os.path.join(path, file_name)
        
        try:
            if os.path.exists(full_path):
                print(f"Lendo {file_name} do ciclo {cycle}")
                df = pd.read_sas(full_path)
                
                # Padronizar nomes das colunas (remover sufixos _B e _C)
                df.columns = [col[:-2] if col.endswith(('_B', '_C')) else col 
                            for col in df.columns]
                
                # Adicionar coluna identificando o ciclo
                df['NHANES_CYCLE'] = cycle
                
                dfs.append(df)
                print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
            else:
                print(f"Arquivo não encontrado: {full_path}")
                return None
        except Exception as e:
            print(f"Erro ao ler {file_name}: {str(e)}")
            return None
    
    if len(dfs) == 3:
        # Encontrar colunas comuns
        common_cols = set.intersection(*[set(df.columns) for df in dfs]) - {'NHANES_CYCLE'}
        columns_to_keep = list(common_cols) + ['NHANES_CYCLE']
        
        # Combinar os datasets
        combined_df = pd.concat([df[columns_to_keep] for df in dfs], 
                              axis=0, ignore_index=True)
        print(f"\nDataset combinado:")
        print(f"Linhas totais: {combined_df.shape[0]}")
        print(f"Colunas: {combined_df.shape[1]}")
        return combined_df
    return None

# Caminhos para os diretórios
base_paths = [
    '/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_1999_2000',
    '/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_2001_2002',
    '/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_2003_2004'
]

# Criar pasta para outputs se ainda não existir
output_dir = os.path.join(os.path.dirname(base_paths[0]), 'combined_datasets')
os.makedirs(output_dir, exist_ok=True)

print("Iniciando combinação dos dados de pânico...")
combined_df = merge_panic_data(base_paths)

if combined_df is not None:
    output_path = os.path.join(output_dir, 'PANIC_combined.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"\nDataset combinado salvo em: {output_path}")
    print("\nPrimeiras colunas do dataset combinado:")
    print(list(combined_df.columns)[:5])
else:
    print("\nNão foi possível combinar os datasets de pânico")

In [ ]:
import pandas as pd

# Caminho para o arquivo combinado
file_path = '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/combined_datasets/PANIC_combined.csv'

# Lista de variáveis para verificar
variables_to_check = [
    'CIQP44A',  # Strong Fear: Giving a speech
    'CIQP44B',  # Strong Fear: Party or social event
    'CIQP44C',  # Strong Fear: Being in a crowd
    'CIQP44D',  # Strong Fear: Meeting new people
    'CIQP44E',  # Strong Fear: Being outside, away
    'CIQP44F',  # Strong Fear: Traveling bus, train, car
    'CIQP44G',  # Strong Fear: Crowd, standing in line
    'CIQP44H'   # Strong Fear: Being in a public place
]

try:
    # Ler o dataset
    df = pd.read_csv(file_path)
    print(f"Total de colunas no dataset: {len(df.columns)}")
    print("\nVerificando variáveis específicas:")
    
    # Verificar cada variável
    for var in variables_to_check:
        if var in df.columns:
            print(f"✓ {var} - Presente")
            # Mostrar alguns valores únicos para confirmar o conteúdo
            unique_values = sorted(df[var].unique())
            print(f"   Valores únicos: {unique_values}")
        else:
            print(f"✗ {var} - Ausente")
    
    print("\nTodas as colunas disponíveis:")
    for col in sorted(df.columns):
        print(f"- {col}")

except Exception as e:
    print(f"Erro ao ler o arquivo: {str(e)}")

In [ ]:
import pandas as pd
import os

# Pasta dos datasets combinados
combined_dir = '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/combined_datasets'

# Verificar cada arquivo CSV
print("Verificando coluna SEQN em todos os datasets combinados:\n")

try:
    for file in sorted(os.listdir(combined_dir)):
        if file.endswith('_combined.csv'):
            file_path = os.path.join(combined_dir, file)
            df = pd.read_csv(file_path)
            
            # Verificar se SEQN está presente
            has_seqn = 'SEQN' in df.columns
            
            print(f"{file:30} - SEQN: {'✓ Presente' if has_seqn else '✗ Ausente'}")
            
            if has_seqn:
                # Verificar se existem valores nulos
                null_count = df['SEQN'].isnull().sum()
                if null_count > 0:
                    print(f"   Atenção: {null_count} valores nulos encontrados!")
                
                # Verificar se os valores são únicos
                unique_count = df['SEQN'].nunique()
                total_count = len(df)
                if unique_count != total_count:
                    print(f"   Atenção: {unique_count} valores únicos em {total_count} linhas")

except Exception as e:
    print(f"Erro ao processar arquivos: {str(e)}")

In [ ]:
import pandas as pd
import os

def merge_all_datasets():
    # Pasta dos datasets combinados
    combined_dir = '/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/combined_datasets'
    
    # Lista para armazenar todos os dataframes
    all_dfs = []
    
    print("Iniciando processo de merge...\n")
    
    # Ler cada arquivo CSV
    for file in sorted(os.listdir(combined_dir)):
        if file.endswith('_combined.csv'):
            print(f"Lendo {file}")
            df = pd.read_csv(os.path.join(combined_dir, file))
            
            # Extrair nome do dataset do nome do arquivo
            dataset_name = file.replace('_combined.csv', '')
            
            # Adicionar prefixo às colunas exceto SEQN e NHANES_CYCLE
            columns_to_rename = [col for col in df.columns if col not in ['SEQN', 'NHANES_CYCLE']]
            df = df.rename(columns={col: f"{dataset_name}_{col}" for col in columns_to_rename})
            
            print(f"- Linhas: {len(df)}")
            print(f"- Colunas: {len(df.columns)}")
            all_dfs.append(df)
    
    # Realizar o merge de todos os dataframes
    print("\nRealizando merge dos datasets...")
    final_df = all_dfs[0]
    
    for i, df in enumerate(all_dfs[1:], 1):
        print(f"\nMerge {i}")
        print(f"- Linhas antes do merge: {len(final_df)}")
        final_df = pd.merge(final_df, df, on=['SEQN', 'NHANES_CYCLE'], how='outer')
        print(f"- Linhas após o merge: {len(final_df)}")
    
    # Estatísticas finais
    print("\nEstatísticas do dataset final:")
    print(f"Total de linhas: {len(final_df)}")
    print(f"Total de colunas: {len(final_df.columns)}")
    
    # Verificar valores nulos
    null_counts = final_df.isnull().sum()
    print("\nColunas com mais valores nulos:")
    print(null_counts[null_counts > 0].sort_values(ascending=False).head())
    
    return final_df

# Executar o merge
print("Iniciando processo de merge completo...")
final_dataset = merge_all_datasets()

# Salvar o resultado
output_path = '/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_complete.csv'
final_dataset.to_csv(output_path, index=False)
print(f"\nDataset completo salvo em: {output_path}")

# Mostrar primeiras linhas
print("\nPrimeiras linhas do dataset final:")
print(final_dataset.head())

In [ ]:
import pandas as pd

# Ler o dataset de pânico para obter os SEQNs
panic_df = pd.read_csv('/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/combined_datasets/PANIC_combined.csv')

# Obter lista única de SEQNs do dataset de pânico
panic_seqns = set(panic_df['SEQN'])
print(f"Número de SEQNs únicos no dataset de pânico: {len(panic_seqns)}")

# Ler o dataset completo
complete_df = pd.read_csv('/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_complete.csv')
print(f"Número de linhas no dataset completo: {len(complete_df)}")

# Criar subset usando apenas os SEQNs do dataset de pânico
subset_df = complete_df[complete_df['SEQN'].isin(panic_seqns)]
print(f"Número de linhas no subset final: {len(subset_df)}")

# Verificar se mantivemos todos os SEQNs do panic
final_seqns = set(subset_df['SEQN'])
missing_seqns = panic_seqns - final_seqns
if missing_seqns:
    print(f"\nAtenção: {len(missing_seqns)} SEQNs do dataset de pânico não foram encontrados no dataset completo")
else:
    print("\nTodos os SEQNs do dataset de pânico foram mantidos no subset")

# Salvar o subset
output_path = '/Users/filipecarvalho/Documents/data_science_projects/SadPred/NHANES_panic_subset.csv'
subset_df.to_csv(output_path, index=False)
print(f"\nSubset salvo em: {output_path}")

# Mostrar algumas estatísticas do subset
print("\nEstatísticas do subset:")
print(f"Número de colunas: {len(subset_df.columns)}")
print("\nDistribuição por ciclo:")
print(subset_df['NHANES_CYCLE'].value_counts())

In [ ]:
import pandas as pd
import numpy as np

# Ler o dataset
df = pd.read_csv("/Users/filipecarvalho/Documents/data_science_projects/PanicPred/NHANES_panic_subset.csv")

# Criar a variável target baseada em CIQGAD_CIDGSCOR = 1
df['target'] = (df['PANIC_CIDPSCOR'] == 1).astype(int)

# Calcular estatísticas gerais
total = len(df)
positivos = df['target'].sum()
negativos = total - positivos

print("Estatísticas da variável target (PANIC_CIDPSCOR = 1):")
print(f"Total de observações: {total}")
print(f"Casos positivos: {positivos} ({(positivos/total)*100:.2f}%)")
print(f"Casos negativos: {negativos} ({(negativos/total)*100:.2f}%)")

# Análise por ciclo
print("\nDistribuição por ciclo NHANES:")
for cycle in df['NHANES_CYCLE'].unique():
    cycle_df = df[df['NHANES_CYCLE'] == cycle]
    cycle_pos = cycle_df['target'].sum()
    cycle_total = len(cycle_df)
    print(f"\n{cycle}:")
    print(f"Total: {cycle_total}")
    print(f"Positivos: {cycle_pos} ({(cycle_pos/cycle_total)*100:.2f}%)")

# Salvar o dataset com a nova variável target
df.to_csv('/Users/filipecarvalho/Documents/data_science_projects/PANICPRED/NHANES_panic_subset_with_target.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import random
import shap
# Ler dados
df = pd.read_csv('NHANES_panic_subset_with_target.csv')

# Lista de variáveis para transformar em categóricas
categorical_vars = ['MCQ_MCQ250C', 'MCQ_MCQ250F', 'DEMO_INDHHINC', 'MCQ_MCQ250A', 
                   'SMQ_SMQ040', 'BPQ_BPQ020', 'BPQ_BPQ060', 'DEMO_DMDEDUC', 
                   'BPQ_BPQ040D', 'MCQ_MCQ160A', 'ALQ_ALQ150', 'PAQ_PAD320']


# Para verificar os tipos atuais
for var in categorical_vars:
    print(f"\nTipo atual de {var}:", df[var].dtype)

# Função auxiliar para fazer comparação segura
def safe_comparison(series, value):
    # Converte para numérico (mantém tanto int como float)
    numeric_series = pd.to_numeric(series, errors='coerce')
    return numeric_series == value

def safe_isin(series, values):
    # Converte para numérico (mantém tanto int como float)
    numeric_series = pd.to_numeric(series, errors='coerce')
    return numeric_series.isin(values)

# Fazer as substituições
df.loc[safe_comparison(df['MCQ_MCQ250C'], 9), 'MCQ_MCQ250C'] = 2

df.loc[safe_isin(df['MCQ_MCQ250F'], [7, 9]), 'MCQ_MCQ250F'] = 2
df.loc[safe_isin(df['MCQ_MCQ250A'], [7, 9]), 'MCQ_MCQ250A'] = 2

# Para DEMO_INDHHINC
df.loc[safe_comparison(df['DEMO_INDHHINC'], 13), 'DEMO_INDHHINC'] = [random.randint(1,4) for _ in range(len(df[safe_comparison(df['DEMO_INDHHINC'], 13)]))]
df.loc[safe_comparison(df['DEMO_INDHHINC'], 12), 'DEMO_INDHHINC'] = [random.randint(5,11) for _ in range(len(df[safe_comparison(df['DEMO_INDHHINC'], 12)]))]
df.loc[safe_isin(df['DEMO_INDHHINC'], [77, 99]), 'DEMO_INDHHINC'] = np.nan

# Para as outras variáveis
df.loc[safe_isin(df['BPQ_BPQ020'], [7, 9]), 'BPQ_BPQ020'] = 2
df.loc[safe_isin(df['BPQ_BPQ060'], [7, 9]), 'BPQ_BPQ060'] = 2

# DEMO_DMDEDUC
df.loc[safe_isin(df['DEMO_DMDEDUC'], [7, 9]), 'DEMO_DMDEDUC'] = [random.randint(1,3) for _ in range(len(df[safe_isin(df['DEMO_DMDEDUC'], [7, 9])]))]

df.loc[safe_isin(df['BPQ_BPQ040D'], [7, 9]), 'BPQ_BPQ040D'] = 2
df.loc[safe_isin(df['MCQ_MCQ160A'], [7, 9]), 'MCQ_MCQ160A'] = 2
df.loc[safe_isin(df['ALQ_ALQ150'], [7, 9]), 'ALQ_ALQ150'] = 2
df.loc[safe_comparison(df['PAQ_PAD320'], 3), 'PAQ_PAD320'] = np.nan

# Para verificar as mudanças
for var in categorical_vars:
    print(f"\nDistribuição de {var}:")
    print(df[var].value_counts(dropna=False))

# Analisar a distribuição por gênero e target
print("Distribuição inicial por gênero e target:")
print(pd.crosstab(df['DEMO_RIAGENDR'], df['target']))

# Separar colunas numéricas e categóricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Pré-processar dados
df_processed = df.copy()

# Imputar valores numéricos coluna por coluna
for col in numeric_cols:
    imputer = SimpleImputer(strategy='median')
    df_processed[col] = imputer.fit_transform(df_processed[[col]])

# Processar dados categóricos
label_encoders = {}
for col in categorical_cols:
    # Primeiro imputar valores faltantes
    df_processed[col] = df_processed[col].fillna('missing')
    # Depois aplicar LabelEncoder
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    label_encoders[col] = le
# Variáveis a remover (mesma lista do código original)
bix_vars_to_remove = ['BIX_BIXC245K', 'BIX_BIXC339K', 'BIX_BIXC083K',
                      "BIX_BIXC200K", "BIX_BIXC067K","BIX_BIXC378K",
                      "BIX_BIXC159K", "BIX_BIXC012K", "BIX_BIXC054K",
                      "BIX_BIXC421K", "BIX_BIXC060K", "BIX_BIXC100K",
                      "BIX_BIXC128K", "BIX_BIXC075K", "BIX_BIXC469K",
                      "BIX_BIXC093K", "BIX_BIXC177K", "BIX_BIXC115K",
                      "BIX_BIXC013K", "BIX_BIXC016K", "BIX_BIXC143K",
                      "BIX_BIXC035K", "BIX_BIXC014K", "BIX_BIXC028K",
                      "BIX_BIXC015K", "BIX_BIXC043K", "BIX_BIXC018K",
                      "BIX_BIXC011K", "BIX_BIXC220K", "BIX_BIXC273K",
                      "BIX_BIXC500K", "BIX_BIXC050K","CIQGAD_CIQG02",
                      "BIX_BIXC020K", "BIX_BIXC806K", "BIX_BIXS005K", 
                      "BIX_BIXC031K", "BIX_BIXC025K", "BIX_BIXC009K",
                      "BIX_BIXC023K", "BIX_BIXC649K", "BIX_BIXC304K", 
                      "BIX_BIXC723K","BIX_BIXC582K","BIX_BIXS013K", "BIX_BIDICF",
                      "BIX_BIXC898K", "MPQ_MPD040","MPQ_MPQ020","MCQ_MCQ180K",
                      "CIQGAD_CIDGSCOR" ,"MPQ_MPQ100","CIQGAD_CIQG01",
                      "CIQGAD_CIQG05","CIQGAD_CIQG33", "CIQGAD_CIQG15", 
                      "CIQGAD_CIQG03", "CIQGAD_CIQG23", "CIQGAD_CIQG30",
                      "CIQGAD_CIQG29", "CIQGAD_CIQG26", "CIQGAD_CIQG16", 
                      "CIQGAD_CIQG06", "CIQGAD_CIQG04", "CIQGAD_CIQG20", 
                       "RXQ_RX_RXDDRGID", "HUQ_HUQ090", "CIQGAD_CIQG28",
                      "CIQGAD_CIQG38", "CIQGAD_CIQG24", "CIQGAD_CIQG35",
                      "CIQGAD_CIQG17G", "CIQGAD_CIQG12", "CIQGAD_CIQG36",
                      "CIQGAD_CIQG09", "BIX_BIXC039K", "BIX_BIXS016K",
                      "HUQ_HUQ050", "CIQGAD_CIQG25", "HUQ_HUQ010", "HSQ_HSQ520",
                       "BPQ_BPQ040B", "MPQ_MPQ110", "MCQ_MCQ245A", "HIQ_HIQ220",
                         "AUQ_AUQ190", "SSMUMP_SSISR", "AUQ_AUQ200", 
                         "CIQGAD_CIQG08", "PFQ_PFQ059", "BIX_BIDTD", "DEMO_DMDHRAGE",
                          "DEMO_DMDHREDU", "ALQ_ALQ110", "BIX_BIDFIT", "BPQ_BPQ040C",
                            "DEMO_DMDHRBRN", "HUQ_HUQ030", "RDQ_RDQ070", "RHQ_RHD080",
                              "RHQ_RHQ210", "RHQ_RHQ250", "BIX_BIXC1M", "CVX_CVDR3HR",
                                "OCQ_OCQ380", "SMQ_SMD100TR", "RHQ_RHQ190", "RHQ_RHQ160",
                    "CVX_CVDEXCL2", "HSQ_HSQ500", "OCQ_OCQ260", "PAQ_PAQ500", "DEMO_SDDSRVYR",
                    "RHQ_RHQ460Q", "UC_URXPREG", "CVX_CVDR3SY", "SMQ_SMD100BR",
                       "BIX_BIDFC", "BPX_BPXSY3", "SMQ_SMD100CO", "MCQ_MCQ245B", "BPX_BPXSAR",
                        "SMQ_SMD100LN", "VIX_VIXKLM2", "SMQ_SMD100NI", "VIX_VIXPLS", "SMQ_SMD130", 
                          "BIX_BIDECF", "BIX_BIAEXSTS", "BPX_BPXSY2", "RHQ_RHQ460U", "RHQ_RHQ460U",
                            "SMQ_SMD100MN", "BIX_BIDTBW", "FSQ_FSD170N", "RHQ_RHQ430",
                              "HUQ_HUQ020", "OCQ_OCD395", "BMX_BMXWAIST", "HSQ_HSQ510",
                                "MPQ_MPQ060", "HOQ_HOD050", "VIX_VIXPRC",
                                  "OHQ_OHQ033",
    "CVX_CVDEXLEN",
    "BIX_BIDALPHA",
    "SMQFAM_SMD430", "BIX_BIXS031K",
    "DIQ_DIQ060U", "VIX_VIXOLSM", "DEMO_RIDEXPRG", "RHQ_RHQ564", "RDQ_RDQ137",
     "DEMO_DMDHRGND", "OHXREF_OHAEXSTS", "CIQGAD_CIQG10", "OCQ_OCD230", "OHQ_OHQ030",
      "BPX_BPXDI4", "HOQ_HOD030", "PH_PHACOFHR", "RHQ_RHQ568Q", "DIQ_DIQ050",
       "DEMO_RIDEXMON", "WHQ_WHQ030", "AUQ_AUQ210", "CVX_CVAPROT", "PH_PHACOFMN", "BPQ_BPQ040F",
        "OHXREF_OHQ134", "CVX_CVDPROT", "VIX_VIXORAM", "OHXREF_OHQ132", "OCQ_OCD480",
         "BIX_BIXC005K", "BPX_BPXSY4", "SXQ_SXQ100", "CIQGAD_CIQG32", "RDQ_RDQ135", "PAQ_PAD080",
          "VIX_VIDROVA", "CVX_CVDS2SY", "MCQ_MCQ180A", "OSQ_OSD030BA", "RHQ_RHQ390", "CIQGAD_CIQG32",
           "SMQ_SMD190", "RHQ_RHQ420", "RHQ_RHQ330", "PH_PHAFSTHR", "RHQ_RHQ180", "MPQ_MPQ090",
            "DEMO_WTMEC2YR", "DEMO_RIDAGEEX", "BIX_BIXS025K", "VIX_VIXKLM1", "VIX_VIXKRM1",
             "CIQGAD_WTSCI2YR", "BIX_BIDPFAT", "VIX_VIXOLCM", "BIX_BIXS035K",
              "BIX_BIXS039K", "VIX_VIXORCM" ]
# Separar features e target 
columns_to_drop = [col for col in df.columns if 'PANIC_' in col] + ['SEQN'] + bix_vars_to_remove
X = df_processed.drop(columns_to_drop + ['target'], axis=1)
y = df_processed['target']

# Primeiro dividir dados em treino e teste para evitar data leakage
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Criar DataFrame de treino completo
train_data = X_train.copy()
train_data['target'] = y_train

# Separar classe minoritária
train_minority = train_data[train_data['target'] == 1]
train_majority = train_data[train_data['target'] == 0]

print("\nDistribuição de gênero na classe minoritária antes do balanceamento:")
print(pd.crosstab(train_minority['DEMO_RIAGENDR'], columns='count'))

# SMOTE para balancear gênero na classe minoritária
minority_males = train_minority[train_minority['DEMO_RIAGENDR'] == 1]
minority_females = train_minority[train_minority['DEMO_RIAGENDR'] == 2]



if len(minority_males) < len(minority_females):
    smote_gender = SMOTE(random_state=42)
    X_minority = train_minority.drop('target', axis=1)
    y_minority_gender = train_minority['DEMO_RIAGENDR']
    
    X_minority_balanced, _ = smote_gender.fit_resample(X_minority, y_minority_gender)
    
    # Adicionar target de volta
    X_minority_balanced['target'] = 1
    
    # Manter apenas as novas amostras necessárias
    n_samples_needed = len(minority_females) - len(minority_males)
    new_samples = X_minority_balanced.tail(n_samples_needed)
    
    # Combinar com dados originais
    train_data_balanced = pd.concat([train_majority, train_minority, new_samples])

print("\nDistribuição de gênero na classe minoritária após balanceamento:")
gender_dist = pd.crosstab(train_data_balanced[train_data_balanced['target']==1]['DEMO_RIAGENDR'], columns='count')
print(gender_dist)

# Preparar dados para o pipeline final
X_train_final = train_data_balanced.drop('target', axis=1)
y_train_final = train_data_balanced['target']

# Criar pipeline final
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(
        n_estimators=200,
        max_depth=5,
        random_state=42,
        min_samples_leaf=1,
        min_samples_split=4,
        learning_rate=0.1
    ))
])

# Treinar o modelo
pipeline.fit(X_train_final, y_train_final)

# Fazer predições
y_pred = pipeline.predict(X_test)

# Imprimir métricas
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Get feature importances
model = pipeline.named_steps['classifier']

# Calcular SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Se o modelo retorna múltiplos outputs (um para cada classe), pegar apenas o da classe positiva
if isinstance(shap_values, list):
    shap_values = shap_values[1]

# Calcular importância absoluta média para cada feature
feature_importance = np.abs(shap_values).mean(0)

# Criar DataFrame com as importâncias SHAP
shap_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance
})
shap_df = shap_df.sort_values('importance', ascending=False)

print("\nTop 30 features mais importantes (SHAP):")
print(shap_df.head(30))

# Salvar para CSV
shap_df.to_csv('shap_importance_gb.csv', index=False)

# Opcional: Criar gráfico de sumário SHAP
shap.summary_plot(shap_values, X_test, plot_size=(10,8))

# Gravar o DataFrame modificado
df.to_csv('NHANES_panic_subset_with_target_modified.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
# Ler dados de importância e selecionar top 23 features
importance_df = pd.read_csv('shap_importance_gb.csv')
top_23_features = importance_df.head(30)['feature'].tolist()

# Criar lista de features incluindo DEMO_RIAGENDR se não estiver nas top 23
features_for_balancing = top_23_features.copy()
if 'DEMO_RIAGENDR' not in features_for_balancing:
    features_for_balancing.append('DEMO_RIAGENDR')

# Ler dataset original
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')


# Separar colunas numéricas e categóricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Pré-processar dados
df_processed = df.copy()

# Imputar valores numéricos coluna por coluna
for col in numeric_cols:
    imputer = SimpleImputer(strategy='median')
    df_processed[col] = imputer.fit_transform(df_processed[[col]])

# Processar dados categóricos
label_encoders = {}
for col in categorical_cols:
    # Primeiro imputar valores faltantes
    df_processed[col] = df_processed[col].fillna('missing')
    # Depois aplicar LabelEncoder
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    label_encoders[col] = le

# Selecionar features para balanceamento e target
X_balance = df_processed[features_for_balancing]
y = df_processed['target']

# Primeiro dividir dados em treino e teste para evitar data leakage
X_train, X_test, y_train, y_test = train_test_split(
    X_balance, y, test_size=0.35, stratify=y, random_state=42
)

# Criar DataFrame de treino completo
train_data = X_train.copy()
train_data['target'] = y_train

# Separar classe minoritária
train_minority = train_data[train_data['target'] == 1]
train_majority = train_data[train_data['target'] == 0]

print("\nDistribuição de gênero na classe minoritária antes do balanceamento:")
print(pd.crosstab(train_minority['DEMO_RIAGENDR'], columns='count'))

# SMOTE para balancear gênero na classe minoritária
minority_males = train_minority[train_minority['DEMO_RIAGENDR'] == 1]
minority_females = train_minority[train_minority['DEMO_RIAGENDR'] == 2]

if len(minority_males) < len(minority_females):
    smote_gender = SMOTE(random_state=42)
    X_minority = train_minority.drop('target', axis=1)
    y_minority_gender = train_minority['DEMO_RIAGENDR']
    
    X_minority_balanced, _ = smote_gender.fit_resample(X_minority, y_minority_gender)
    
    # Adicionar target de volta
    X_minority_balanced['target'] = 1
    
    # Manter apenas as novas amostras necessárias
    n_samples_needed = len(minority_females) - len(minority_males)
    new_samples = X_minority_balanced.tail(n_samples_needed)
    
    # Combinar com dados originais
    train_data_balanced = pd.concat([train_majority, train_minority, new_samples])

print("\nDistribuição de gênero na classe minoritária após balanceamento:")
gender_dist = pd.crosstab(train_data_balanced[train_data_balanced['target']==1]['DEMO_RIAGENDR'], columns='count')
print(gender_dist)

# Preparar dados para o pipeline final 

#['SXQ_SXQ120', 'RHQ_RHQ370', 'MCQ_MCQ180B' HUQ_HUQ060

X_train_final = train_data_balanced.drop(['target',"DEMO_RIAGENDR",
    "OCQ_OCD390",
    "SXQ_SXQ020",
    "MCQ_MCQ250C",
    "AUQ_AUQ130",
    "BPQ_BPQ040D",
    "MCQ_MCQ230A",
    "PH_PHAFSTMN", "SSMUMP_WTSSMC2Y", "BMX_BMXSUB",
    "AUQ_AUQ220"] if 'DEMO_RIAGENDR' not in top_23_features else ['target'], axis=1)
y_train_final = train_data_balanced['target']

# Ajustar X_test também

X_test_final = X_test.drop(['DEMO_RIAGENDR',
    "OCQ_OCD390",
    "SXQ_SXQ020",
    "MCQ_MCQ250C",
    "AUQ_AUQ130",
    "BPQ_BPQ040D",
    "MCQ_MCQ230A",
    "PH_PHAFSTMN", "SSMUMP_WTSSMC2Y", "BMX_BMXSUB",
    "AUQ_AUQ220"], axis=1) if 'DEMO_RIAGENDR' not in top_23_features else X_test

# Criar pipeline final
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(
        n_estimators=200,
        max_depth=8,
        random_state=42,
        min_samples_leaf=1,
        min_samples_split=4,
        learning_rate=0.1
    ))
])

# Treinar o modelo
pipeline.fit(X_train_final, y_train_final)

# Fazer predições
y_pred = pipeline.predict(X_test_final)

# Imprimir métricas
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Get feature importances
model = pipeline.named_steps['classifier']

# Create importance dataframe
importance_df_new = pd.DataFrame({
    'feature': X_train_final.columns,
    'importance': model.feature_importances_
})
importance_df_new = importance_df_new.sort_values('importance', ascending=False)

print("\nTop features importances:")
print(importance_df_new)

# Save to CSV
importance_df_new.to_csv('feature_importance_gb_new.csv', index=False)

# Create visualization of new feature importances
plt.figure(figsize=(15, 10))
plt.barh(range(len(importance_df_new)), importance_df_new['importance']*100)
plt.yticks(range(len(importance_df_new)), importance_df_new['feature'])
plt.xlabel('Importance (%)')
plt.title('Feature Importance Distribution (33 Features)')
plt.tight_layout()
plt.savefig('feature_importance_33features.png')
plt.show()

# Criar Confusion Matrix
plt.figure(figsize=(10,7))
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no panic', 'With panic'])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix_33features.png')
plt.show()

In [ ]:
df["target"].value_counts()

In [ ]:
import pandas as pd
import numpy as np

# Carregar o dataset
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')

# Mapeamentos para variáveis categóricas
category_maps = {
    'DEMO_RIAGENDR': {1: 'Male', 2: 'Female'},
    'DEMO_DMDEDUC': {
        1: 'Less Than High School', 
        2: 'High School Diploma', 
        3: 'More Than High School'
    },
    'DEMO_INDHHINC': {
        1: '$0 to $4,999',  # Combina 1 e 2
        3: '$5,000 to $9,999',  # Combina 3 e 4
        5: '$10,000 to $14,999',  # Combina 5 e 6
        7: '$15,000 to $19,999',  # Combina 7 e 8
        11: '$20,000 to $34,999',  # Usa categoria existente
        12: '$35,000 to $54,999',  # Combina algumas categorias
        16: '$55,000 and Over'  # Mantém categoria existente
    }
}

# Função para calcular estatísticas com percentagens dentro do grupo
def calculate_group_stats(df, var, is_categorical=False):
    # Converter coluna para tipo categórico, tratando NaNs
    if is_categorical:
        # Filtrar categorias indesejadas para variáveis específicas
        if var == 'DEMO_DMDEDUC':
            df = df[df[var].isin([1, 2, 3])]
        elif var == 'DEMO_INDHHINC':
            # Mapear valores para novos intervalos
            income_mapping = {
                1: '$0 to $4,999',
                2: '$0 to $4,999',
                3: '$5,000 to $9,999',
                4: '$5,000 to $9,999',
                5: '$10,000 to $14,999',
                6: '$10,000 to $14,999',
                7: '$15,000 to $19,999',
                8: '$15,000 to $19,999',
                9: '$20,000 to $34,999',
                10: '$20,000 to $34,999',
                11: '$20,000 to $34,999',
                12: '$35,000 to $54,999',
                13: '$35,000 to $54,999',
                14: '$35,000 to $54,999',
                15: '$55,000 and Over',
                16: '$55,000 and Over'
            }
            df['DEMO_INDHHINC_grouped'] = df['DEMO_INDHHINC'].map(income_mapping)
            var = 'DEMO_INDHHINC_grouped'
        
        # Converter para categoria, tratando NaNs
        df[var] = df[var].fillna('Unknown').astype('category')
        
        # Contagem total e percentagem global
        total_counts = df[var].value_counts()
        total_percents = total_counts / len(df) * 100
        
        # Estatísticas para grupo normal
        normal_counts = df[df['target'] == 0][var].value_counts()
        normal_percents = normal_counts / len(df[df['target'] == 0]) * 100
        
        # Estatísticas para grupo com pânico
        panic_counts = df[df['target'] == 1][var].value_counts()
        panic_percents = panic_counts / len(df[df['target'] == 1]) * 100
        
        # Combinar resultados
        results = []
        for category in df[var].cat.categories:
            # Mapear código numérico para label descritiva
            category_label = category_maps.get(var, {}).get(category, str(category))
            results.append([
                category_label,
                f"{total_counts.get(category, 0)} ({total_percents.get(category, 0):.1f}%)",
                f"{normal_counts.get(category, 0)} ({normal_percents.get(category, 0):.1f}%)",
                f"{panic_counts.get(category, 0)} ({panic_percents.get(category, 0):.1f}%)"
            ])
        return results
    else:
        # Para variáveis contínuas
        total_mean = df[var].mean()
        total_std = df[var].std()
        
        normal_mean = df[df['target'] == 0][var].mean()
        normal_std = df[df['target'] == 0][var].std()
        
        panic_mean = df[df['target'] == 1][var].mean()
        panic_std = df[df['target'] == 1][var].std()
        
        return [
            f"{total_mean:.1f} ± {total_std:.1f}",
            f"{normal_mean:.1f} ± {normal_std:.1f}",
            f"{panic_mean:.1f} ± {panic_std:.1f}"
        ]

# Variáveis a analisar
continuous_vars = {
    'DEMO_RIDAGEEX': 'Age (years)',
    'BMX_BMXWT': 'Weight (kg)',
    'BMX_BMXHT': 'Height (cm)',
    'BMX_BMXBMI': 'BMI'
}

categorical_vars = {
    'DEMO_RIAGENDR': 'Gender',
    'DEMO_DMDEDUC': 'Education Level',
    'DEMO_INDHHINC': 'Household Income'
}

# Gerar resultados
print("Population Characteristics:")

# Variáveis contínuas
for var, label in continuous_vars.items():
    if var == 'DEMO_RIDAGEEX':
        # Converter de meses para anos
        df[var] = df[var] / 12
    
    stats = calculate_group_stats(df, var)
    print(f"\n{label}:")
    print(f"Total: {stats[0]}")
    print(f"Normal: {stats[1]}")
    print(f"Panic Group: {stats[2]}")

# Variáveis categóricas
for var, label in categorical_vars.items():
    print(f"\n{label}:")
    results = calculate_group_stats(df, var, is_categorical=True)
    for row in results:
        print(row)

# Estatísticas adicionais
print("\nAdditional Statistics:")
print(f"Total Sample Size: {len(df)}")
print(f"Normal Group Size: {len(df[df['target'] == 0])}")
print(f"Panic Group Size: {len(df[df['target'] == 1])}")
print(f"Panic Group Prevalence: {len(df[df['target'] == 1])/len(df)*100:.2f}%")

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# Carregar o dataset
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')

# Função para calcular estatísticas com percentagens dentro do grupo
def calculate_group_stats(df, var, is_categorical=False):
    # Estatísticas para todo o dataset
    total_stats = []
    
    if is_categorical:
        # Contagem total e percentagem global
        total_counts = df[var].value_counts()
        total_percents = total_counts / len(df) * 100
        
        # Estatísticas para grupo normal
        normal_counts = df[df['target'] == 0][var].value_counts()
        normal_percents = normal_counts / len(df[df['target'] == 0]) * 100
        
        # Estatísticas para grupo com pânico
        panic_counts = df[df['target'] == 1][var].value_counts()
        panic_percents = panic_counts / len(df[df['target'] == 1]) * 100
        
        # Combinar resultados
        results = []
        for category in sorted(df[var].unique()):
            results.append([
                f"{category}",
                f"{total_counts.get(category, 0)} ({total_percents.get(category, 0):.1f}%)",
                f"{normal_counts.get(category, 0)} ({normal_percents.get(category, 0):.1f}%)",
                f"{panic_counts.get(category, 0)} ({panic_percents.get(category, 0):.1f}%)"
            ])
        return results
    else:
        # Para variáveis contínuas
        total_mean = df[var].mean()
        total_std = df[var].std()
        
        normal_mean = df[df['target'] == 0][var].mean()
        normal_std = df[df['target'] == 0][var].std()
        
        panic_mean = df[df['target'] == 1][var].mean()
        panic_std = df[df['target'] == 1][var].std()
        
        return [
            f"{total_mean:.1f} ± {total_std:.1f}",
            f"{normal_mean:.1f} ± {normal_std:.1f}",
            f"{panic_mean:.1f} ± {panic_std:.1f}"
        ]

# Variáveis a analisar
categorical_vars = {
    'DEMO_RIAGENDR': 'Gender',
    'DEMO_DMDEDUC': 'Education Level',
    'DEMO_INDHHINC': 'Household Income'
}

continuous_vars = {
    'DEMO_RIDAGEEX': 'Age (years)',
    'BMX_BMXWT': 'Weight (kg)',
    'BMX_BMXHT': 'Height (cm)',
    'BMX_BMXBMI': 'BMI'
}

# Gerar resultados
print("Population Characteristics:")

# Variáveis contínuas
for var, label in continuous_vars.items():
    divisor = 12 if var == 'DEMO_RIDAGEEX' else 1
    stats = calculate_group_stats(df, var)
    print(f"\n{label}:")
    print(f"Total: {stats[0]}")
    print(f"Normal: {stats[1]}")
    print(f"Panic Group: {stats[2]}")

# Variáveis categóricas
for var, label in categorical_vars.items():
    print(f"\n{label}:")
    results = calculate_group_stats(df, var, is_categorical=True)
    for row in results:
        print(row)

# Estatísticas adicionais
print("\nAdditional Statistics:")
print(f"Total Sample Size: {len(df)}")
print(f"Normal Group Size: {len(df[df['target'] == 0])}")
print(f"Panic Group Size: {len(df[df['target'] == 1])}")
print(f"Panic Group Prevalence: {len(df[df['target'] == 1])/len(df)*100:.2f}%")

In [ ]:
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')

df["target"].value_counts()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar o dataset
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')

# Converter idade de meses para anos
df['Age'] = df['DEMO_RIDAGEEX'] / 12

# Configurar o estilo do gráfico
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Criar boxplot
sns.boxplot(x='target', y='Age', data=df, 
          palette=['skyblue', 'lightcoral'],
          medianprops={"color": "darkred", "linewidth": 2},
          boxprops={"edgecolor": "navy", "alpha": 0.7},
          whiskerprops={"color": "navy", "linestyle": "--"})

# Personalizar o gráfico
plt.title('Age Distribution: Normal vs Panic Group', fontsize=16, fontweight='bold')
plt.xlabel('Group', fontsize=12)
plt.ylabel('Age (years)', fontsize=12)
plt.xticks([0, 1], ['Normal', 'Panic Group'])

# Adicionar descrição estatística (atualize os valores conforme necessário)
plt.text(0.02, 0.95, 'Normal Group\nMean: 29.5\nStd: 5.8', 
       transform=plt.gca().transAxes, 
       verticalalignment='top',
       bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
plt.text(0.7, 0.95, 'Panic Group\nMean: 31.1\nStd: 4.7', 
       transform=plt.gca().transAxes, 
       verticalalignment='top',
       bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()

In [ ]:
# Após criar o boxplot, adicione:
# Criar mini-tabela de estatísticas
stats_table = pd.DataFrame({
   'Group': ['Normal', 'Panic Group'],
   'Mean Age': [
       df[df['target'] == 0]['Age'].mean(),
       df[df['target'] == 1]['Age'].mean()
   ],
   'Std Dev': [
       df[df['target'] == 0]['Age'].std(),
       df[df['target'] == 1]['Age'].std()
   ],
   '25th Percentile': [
       df[df['target'] == 0]['Age'].quantile(0.25),
       df[df['target'] == 1]['Age'].quantile(0.25)
   ],
   '75th Percentile': [
       df[df['target'] == 0]['Age'].quantile(0.75),
       df[df['target'] == 1]['Age'].quantile(0.75)
   ]
})

# Formatar valores numéricos para 2 casas decimais
stats_table = stats_table.round(2)

# Exibir mini-tabela
print("\nAge Distribution Statistics:")
print(stats_table.to_string(index=False))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar o dataset
df = pd.read_csv('NHANES_panic_subset_with_target_modified.csv')

# Mapeamentos para labels
gender_map = {1: 'Male', 2: 'Female'}
education_map = {
   1: 'Less Than High School', 
   2: 'High School Diploma', 
   3: 'More Than High School'
}

# Preparar dados para gráfico
def prepare_categorical_data(column, mapping):
   df[f'{column}_mapped'] = df[column].map(mapping)
   proportions = df.groupby(['target', f'{column}_mapped']).size().unstack(fill_value=0)
   proportions_percent = proportions.apply(lambda x: x / x.sum() * 100, axis=1)
   return proportions_percent

# Configurações de estilo
plt.figure(figsize=(16, 7), dpi=300)
plt.rcParams.update({'font.size': 18})

# Gráfico de Gênero
plt.subplot(1, 2, 1)
gender_props = prepare_categorical_data('DEMO_RIAGENDR', gender_map)
gender_props.plot(kind='bar', ax=plt.gca(), width=0.8)
plt.title('A. Gender Distribution', fontsize=20, fontweight='bold')
plt.xlabel('Group', fontsize=18)
plt.ylabel('Percentage', fontsize=18)
plt.legend(title='Gender', fontsize=16, title_fontsize=16)
plt.xticks([0, 1], ['Normal', 'Panic'] ,fontsize=14, rotation=0)

# Gráfico de Educação
plt.subplot(1, 2, 2)
education_props = prepare_categorical_data('DEMO_DMDEDUC', education_map)
education_props = education_props[['Less Than High School', 'High School Diploma', 'More Than High School']]
education_props.plot(kind='bar', ax=plt.gca(), width=0.8)
plt.title('B. Education Level Distribution', fontsize=20, fontweight='bold')
plt.xlabel('Group', fontsize=18)
plt.ylabel('Percentage', fontsize=18)
plt.legend(title='Education', fontsize=16, title_fontsize=16)
plt.xticks([0, 1], ['Normal', 'Panic'],fontsize=14, rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

def evaluate_and_plot_model(pipeline, X_train, y_train, X_test, y_test, n_splits=5, random_state=42):
    """
    Evaluate the model using cross-validation on training data and final evaluation on test data.
    """
    # Set up cross-validation
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Perform cross-validation on training data
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Final evaluation on test data
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    results = {
        'cv_scores': cv_scores,
        'cv_mean_score': cv_scores.mean(),
        'cv_std_score': cv_scores.std(),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'probabilities': y_pred_proba,
        'y_true': y_test,
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    # Plotting
    plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 16,
        'axes.labelsize': 14,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'legend.fontsize': 14
    })
    
    fig = plt.figure(figsize=(20, 15))
    gs = fig.add_gridspec(3, 2)
    
    # 1. Cross-validation Results
    ax_cv = fig.add_subplot(gs[0, :])
    cv_data = pd.DataFrame({'Fold': [f'Fold {i+1}' for i in range(n_splits)], 'ROC AUC': results['cv_scores']})
    sns.barplot(data=cv_data, x='Fold', y='ROC AUC', ax=ax_cv, color='skyblue')
    ax_cv.axhline(y=results['cv_mean_score'], color='red', linestyle='--', 
                  label=f'Mean CV Score: {results["cv_mean_score"]:.3f}')
    ax_cv.fill_between([-0.5, n_splits-0.5], 
                      results['cv_mean_score'] - 2*results['cv_std_score'],
                      results['cv_mean_score'] + 2*results['cv_std_score'],
                      alpha=0.2, color='red',
                      label=f'±2 STD: {2*results["cv_std_score"]:.3f}')
    ax_cv.set_title('Cross-validation Results by Fold', fontsize=18, pad=20)
    ax_cv.set_ylim([min(results['cv_scores'])-0.05, max(results['cv_scores'])+0.05])
    ax_cv.legend(fontsize=14)
    
    # 2. Class Accuracy
    ax_acc = fig.add_subplot(gs[1, 0])
    tn, fp, fn, tp = results['confusion_matrix'].ravel()
    accuracy_0 = tn / (tn + fp)
    accuracy_1 = tp / (tp + fn)
    performance_data = pd.DataFrame({
        'Class': ['Negative (0)', 'Positive (1)'],
        'Accuracy': [accuracy_0, accuracy_1]
    })
    sns.barplot(data=performance_data, x='Class', y='Accuracy', ax=ax_acc)
    ax_acc.set_title('Accuracy by Class', fontsize=18)
    ax_acc.set_ylim([0, 1])
    for i, v in enumerate(performance_data['Accuracy']):
        ax_acc.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')
    
    # 3. Confusion Matrix
    ax_cm = fig.add_subplot(gs[1, 1])
    sns.heatmap(results['confusion_matrix'], annot=True, fmt='d', cmap='Blues', 
                ax=ax_cm, annot_kws={'size': 14})
    ax_cm.set_title('Confusion Matrix', fontsize=18)
    ax_cm.set_xlabel('Predicted', fontsize=14)
    ax_cm.set_ylabel('Actual', fontsize=14)
    
    # 4. Probability Distribution
    ax_prob = fig.add_subplot(gs[2, 0])
    sns.histplot(data=pd.DataFrame({
        'Probability': results['probabilities'],
        'True Class': results['y_true']
    }), x='Probability', hue='True Class', bins=30, ax=ax_prob)
    ax_prob.set_title('Prediction Probability Distribution', fontsize=18)
    
    # 5. ROC Curve
    ax_roc = fig.add_subplot(gs[2, 1])
    fpr, tpr, _ = roc_curve(results['y_true'], results['probabilities'])
    ax_roc.plot(fpr, tpr, linewidth=2)
    ax_roc.plot([0, 1], [0, 1], 'k--', linewidth=2)
    ax_roc.set_title(f'ROC Curve (AUC = {results["roc_auc"]:.3f})', fontsize=18)
    ax_roc.set_xlabel('False Positive Rate', fontsize=14)
    ax_roc.set_ylabel('True Positive Rate', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    return results

# Uso da função
results_panic = evaluate_and_plot_model(pipeline, X_train_final, y_train_final, X_test_final, y_test)

In [ ]:
# Extrair informações relevantes para a figure caption
cv_mean = results_panic['cv_mean_score']
cv_std = results_panic['cv_std_score']
test_auc = results_panic['roc_auc']
tn, fp, fn, tp = results_panic['confusion_matrix'].ravel()
accuracy_0 = tn / (tn + fp)
accuracy_1 = tp / (tp + fn)
overall_accuracy = (tn + tp) / (tn + fp + fn + tp)

# Informações sobre a distribuição de probabilidades
probabilities = results_panic['probabilities']
mean_prob = np.mean(probabilities)
median_prob = np.median(probabilities)

# Imprimir as informações
print(f"Cross-validation mean ROC AUC: {cv_mean:.3f} ± {cv_std:.3f}")
print(f"Test set ROC AUC: {test_auc:.3f}")
print(f"Accuracy for class 0 (Negative): {accuracy_0:.3f}")
print(f"Accuracy for class 1 (Positive): {accuracy_1:.3f}")
print(f"Overall accuracy: {overall_accuracy:.3f}")
print(f"Confusion Matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}")
print(f"Mean probability: {mean_prob:.3f}")
print(f"Median probability: {median_prob:.3f}")

In [ ]:
import shap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Certificar-se de usar apenas features numéricas
X_explain = X_test_final.select_dtypes(include=[np.number])

# Criar explicador SHAP
model = pipeline.named_steps['classifier']
explainer = shap.TreeExplainer(model)

# Calcular SHAP values
shap_values = explainer.shap_values(X_explain)

# Garantir que estamos usando os valores da classe positiva para binário
if isinstance(shap_values, list):
    shap_values = shap_values[1]

# 1. Salvar valores de SHAP
pd.DataFrame(
    shap_values, 
    columns=X_explain.columns
).to_csv('shap_values.csv', index=False)

# 2. Gerar summary plot
plt.figure(figsize=(15, 10))
shap.summary_plot(shap_values, X_explain, 
                  feature_names=X_explain.columns.tolist(), 
                  show=False)
plt.title('SHAP Summary Plot for Panic Disorder Prediction')
plt.tight_layout()
plt.savefig('shap_summary_plot.png')
plt.close()

# 3. Gerar bar plot de importância
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_explain, 
                  plot_type="bar", 
                  feature_names=X_explain.columns.tolist(), 
                  show=False)
plt.title('SHAP Feature Importance')
plt.tight_layout()
plt.savefig('shap_feature_importance.png')
plt.close()

# 4. Calcular importância média das features
shap_importance = np.abs(shap_values).mean(axis=0)
shap_importance_df = pd.DataFrame({
    'feature': X_explain.columns,
    'mean_shap_importance': shap_importance
}).sort_values('mean_shap_importance', ascending=False)

shap_importance_df.to_csv('shap_feature_importance.csv', index=False)

print("Arquivos gerados:")
print("1. shap_values.csv")
print("2. shap_summary_plot.png")
print("3. shap_feature_importance.png")
print("4. shap_feature_importance.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import numpy as np

# Configurações de estilo
plt.style.use('seaborn')

# Criar figura com subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 20))
fig.suptitle('SHAP Analysis for Panic Disorder Prediction', fontsize=16, fontweight='bold')

data = {
    "Variable": [
        "CVX_CVDEXSTS",
        "WHQ_WHD120",
        "BIX_BIDFAT",
        "SMQ_SMD030",
        "BMX_BMXTRI",
        "DEMO_INDFMINC",
        "BPX_PEASCTM1",
        "DEMO_INDFMPIR",
        "IMQ_IMQ020",
        "BPX_BPXDAR",
        "WHQ_WHD050",
        "DEMO_RIDAGEMN",
        "MPQ_MPQ070",
        "BPQ_BPQ010",
        "ALQ_ALQ150",
        "MCQ_MCQ250F"
    ],
    "Category": [
        "Cardiovascular Fitness",
        "Weight History",
        "Body Composition",
        "Smoking - Cigarette/Tobacco Use",
        "Body Measures",
        "Demographics - Socioeconomic",
        "Blood Pressure",
        "Demographics - Socioeconomic",
        "Immunization",
        "Blood Pressure",
        "Weight History",
        "Demographics - Age",
        "Pain",
        "Blood Pressure & Cholesterol",
        "Alcohol Use",
        "Medical Conditions - Family History"
    ]
}

# SHAP Analysis
# Garantir que todos os dados sejam numéricos
X_explain = X_test_final.select_dtypes(include=[np.number])

# Criar explicador SHAP
model = pipeline.named_steps['classifier']
explainer = shap.TreeExplainer(model)

# Calcular SHAP values
shap_values = explainer.shap_values(X_explain)
if isinstance(shap_values, list):
   shap_values = shap_values[1]  # Selecionar classe positiva

# Calcular importância média de SHAP
shap_importance = np.abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({
   'Variable': X_explain.columns,
   'SHAP_Importance': shap_importance
})

# Adicionar categoria ao DataFrame SHAP
shap_df['Category'] = shap_df['Variable'].map(dict(zip(data['Variable'], data['Category'])))

# Calcular média de SHAP por categoria
category_shap_importance = shap_df.groupby('Category')['SHAP_Importance'].max()

# Ordenar categorias por sua máxima importância SHAP
sorted_categories = category_shap_importance.sort_values(ascending=False).index

# Preparar DataFrame para plotagem
shap_df_sorted = []
for category in sorted_categories:
   category_df = shap_df[shap_df['Category'] == category].sort_values('SHAP_Importance', ascending=False)
   shap_df_sorted.append(category_df)

# Concatenar dataframes
shap_df_final = pd.concat(shap_df_sorted)

# 1. SHAP Feature Importance por Categoria
sns.barplot(
   data=shap_df_final,
   x="SHAP_Importance",
   y="Variable",
   hue="Category",
   dodge=False,
   palette="tab10",
   ax=ax1
)
ax1.set_title('SHAP Feature Importance by Category', fontsize=14)
ax1.set_xlabel('Mean |SHAP Value|', fontsize=12)
ax1.set_ylabel('Variables', fontsize=12)
handles, _ = ax1.get_legend_handles_labels()
ax1.legend(handles, shap_df_final['Category'].unique(), title="Category", bbox_to_anchor=(1.05, 1), loc='upper left')

# 2. SHAP Summary Plot 
plt.sca(ax2)
shap.summary_plot(shap_values, X_explain, 
                 feature_names=X_explain.columns.tolist(),
                 plot_type='dot',
                 show=False)
ax2.set_title('SHAP Summary Plot', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Primeiro gráfico: SHAP Feature Importance por Categoria
plt.figure(figsize=(12, 8))
sns.barplot(
   data=shap_df_final,
   x="SHAP_Importance",
   y="Variable",
   hue="Category",
   dodge=False,
   palette="tab10"
)
plt.title('SHAP Feature Importance by Category', fontsize=14, pad=20)
plt.xlabel('Mean |SHAP Value|', fontsize=12)
plt.ylabel('Variables', fontsize=12)
handles, _ = plt.gca().get_legend_handles_labels()
plt.legend(handles, shap_df_final['Category'].unique(), 
         title="Category", 
         bbox_to_anchor=(1.05, 1), 
         loc='upper left')
plt.tight_layout()
plt.show()

# Segundo gráfico: SHAP Summary Plot
plt.figure(figsize=(12, 8))
shap.summary_plot(
   shap_values, 
   X_explain,
   feature_names=X_explain.columns.tolist(),
   plot_type='dot',
   show=True
)
plt.title('SHAP Summary Plot', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
def generate_shap_analysis_report(shap_df_final, shap_values, X_explain):
    report = "# SHAP Analysis Report for Panic Disorder Prediction\n\n"
    
    # Overview
    report += "## Overview\n"
    report += "This analysis explores the feature importance and impact in predicting panic disorder using SHAP (SHapley Additive exPlanations) values, derived from a Gradient Boosting Classifier trained on NHANES 1999-2004 data.\n\n"
    
    # Feature Importance by Category
    report += "### SHAP Feature Importance by Category\n\n"
    report += "#### Top Categories Influencing Panic Disorder Risk:\n"
    
    # Calculate category importance
    category_importance = shap_df_final.groupby('Category')['SHAP_Importance'].mean().sort_values(ascending=False)
    
    for i, (category, importance) in enumerate(category_importance.items(), 1):
        report += f"{i}. **{category}**\n"
        
        # Get top features for this category
        category_features = shap_df_final[shap_df_final['Category'] == category].sort_values('SHAP_Importance', ascending=False)
        top_features = category_features.head(3)
        
        report += f"   - Top features: {', '.join(top_features['Variable'])}\n"
        report += f"   - Mean SHAP Importance: {importance:.4f}\n\n"
    
    # Detailed Insights
    report += "#### Detailed Insights:\n"
    report += "- Top individual features demonstrate complex, multifaceted risk factors\n"
    report += "- Medical history appears more predictive than single lifestyle metrics\n"
    report += "- Physiological and behavioral factors interact in panic disorder prediction\n\n"
    
    # SHAP Summary Plot Interpretation
    report += "### SHAP Summary Plot Interpretation\n\n"
    report += "#### Feature Impact Characteristics:\n"
    report += "- Color indicates feature's impact direction\n"
    report += "- Red: Increased panic disorder risk\n"
    report += "- Blue: Decreased panic disorder risk\n"
    report += "- Horizontal spread shows magnitude of impact\n\n"
    
    # Key Observations
    report += "#### Key Observations:\n"
    report += "- Most features show nuanced, non-linear relationships with panic disorder risk\n"
    report += "- Some features demonstrate high variability in their predictive power\n"
    report += "- No single feature definitively determines panic disorder risk\n\n"
    
    # Additional Statistical Details
    report += "### Additional Statistical Details\n"
    report += f"- Total Features Analyzed: {len(X_explain.columns)}\n"
    report += f"- Top 5 Most Important Features:\n"
    
    top_5_features = shap_df_final.sort_values('SHAP_Importance', ascending=False).head(5)
    for _, row in top_5_features.iterrows():
        report += f"  1. **{row['Variable']}** (Category: {row['Category']}): {row['SHAP_Importance']:.4f}\n"
    
    # Methodological Notes
    report += "\n### Methodological Considerations\n"
    report += "- Analysis based on machine learning model trained on NHANES dataset\n"
    report += "- SHAP values provide model-agnostic feature importance\n"
    report += "- Captures complex interactions beyond traditional feature importance methods\n\n"
    
    # Limitations
    report += "### Limitations\n"
    report += "- Observational study design\n"
    report += "- Potential confounding factors\n"
    report += "- Requires validation in diverse populations\n\n"
    
    # Recommendations
    report += "### Recommendations\n"
    report += "1. Further investigate top predictive medical and physiological factors\n"
    report += "2. Consider personalized risk assessment incorporating multiple factors\n"
    report += "3. Develop targeted interventions addressing identified risk patterns\n"
    
    return report

# Usar a função
report = generate_shap_analysis_report(shap_df_final, shap_values, X_explain)

# Salvar relatório em arquivo
with open('shap_analysis_report.md', 'w') as f:
    f.write(report)

# Opcional: imprimir relatório
print(report)

In [ ]:
def plot_shap_summary(shap_values, X_explain, max_display=12):
    # Calcular médias absolutas dos valores SHAP para cada feature
    feature_importance = np.abs(shap_values).mean(0)
    feature_names = X_explain.columns.tolist()
    
    # Criar DataFrame com os valores SHAP
    plot_data = []
    for feature_idx in range(len(feature_names)):
        feature_shap_values = shap_values[:, feature_idx]
        feature_values = X_explain.iloc[:, feature_idx]
        
        for i in range(len(feature_shap_values)):
            plot_data.append({
                'Feature': feature_names[feature_idx],
                'SHAP Value': feature_shap_values[i],
                'Feature Value': feature_values[i],
                'Importance': feature_importance[feature_idx]
            })
    
    df_plot = pd.DataFrame(plot_data)
    
    # Ordenar features por importância
    top_features = (df_plot.groupby('Feature')['Importance']
                   .mean()
                   .sort_values(ascending=True)
                   .tail(max_display)
                   .index)
    
    df_plot = df_plot[df_plot['Feature'].isin(top_features)]
    
    # Criar gráfico
    fig = px.strip(
        df_plot,
        y='Feature',
        x='SHAP Value',
        color='Feature Value',
        title='SHAP Summary Plot',
        width=1000,  # Aumentar largura
        height=600,  # Aumentar altura
    )
    
    # Customizar layout
    fig.update_layout(
        plot_bgcolor='white',
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray',
            tickfont=dict(size=12),  # Aumentar tamanho da fonte
            showline=True,
            linewidth=1,
            linecolor='black'
        ),
        xaxis=dict(
            title='SHAP Value',
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray',
            zeroline=True,
            zerolinewidth=1.5,
            zerolinecolor='black',
            showline=True,
            linewidth=1,
            linecolor='black'
        ),
        margin=dict(l=200),  # Aumentar margem esquerda
        showlegend=False,
        height=30 * len(top_features) + 150  # Altura dinâmica baseada no número de features
    )
    
    # Adicionar linhas de grade horizontais mais visíveis
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    return fig

In [ ]:
df_shap= pd.read_csv("/Users/filipecarvalho/Documents/data_science_projects/PanicPred/shap_feature_importance.csv" )

In [ ]:
print(df_shap.head(20))